### 0. Import Libs

In [1]:
# kernel must be msd2
import pandas as pd
import geopandas as gpd
from shapely.geometry import *
import sys
from shapely import wkt
sys.path.append('../')
from util.epsg_transform import *
import constant as c
import os
import matplotlib.pyplot as plt

In [2]:
# Get df_bd
df_bd = pd.read_csv('../asset/preprocess/features/df_bd.csv', index_col=0, dtype={'mainpurpscd': str})
df_bd.dropna(subset=['x', 'y'], inplace=True)
x_5179, y_5179 = coord_tranformation(list(df_bd.x), list(df_bd.y), "epsg:4326", "epsg:5179")
df_bd['x_5179'] = x_5179
df_bd['y_5179'] = y_5179
df_bd['geometry'] = df_bd.apply(lambda row: Point([row['x_5179'], row['y_5179']]), axis = 1)
df_bd = gpd.GeoDataFrame(df_bd, geometry='geometry', crs = 'EPSG:5179')

# Get df_grid
df_grid = pd.read_csv('../asset/preprocess/grid/df_grid.csv', index_col = 0)
df_grid = df_grid.rename(columns = {'geometry': 'geometry_grids'})
df_grid['geometry_grids'] = df_grid['geometry_grids'].apply(wkt.loads)
df_grid = df_grid[['idx', 'ADM_SECT_C', 'geometry_grids']]
df_grid = df_grid.rename(columns={'idx': 'grid_idx'})
df_grid = gpd.GeoDataFrame(df_grid, crs = 'EPSG:5179', geometry = 'geometry_grids')
df_grid = df_grid.to_crs({'init':'epsg:5179'})
df_grid['grid_idx'] = df_grid['grid_idx'].astype(str)

# For given bd info(coord x,y) find grid that x, y falles in and assign
df_bd_assigned = gpd.sjoin(df_bd, df_grid, op = 'within', how = 'left')
df_bd_assigned = df_bd_assigned[~df_bd_assigned['grid_idx'].isna()]
df_bd_assigned = df_bd_assigned.drop('index_right', axis=1)
df_bd_assigned['grid_idx'] = df_bd_assigned['grid_idx'].astype(float).astype(int).astype(str)
df_bd_assigned['ADM_SECT_C'] = df_bd_assigned['ADM_SECT_C'].astype(float).astype(int).astype(str)

# Concat dfs
df_grid_feature = df_bd_assigned.merge(df_grid, on = 'grid_idx')
df_grid_feature = df_grid_feature.drop('ADM_SECT_C_y', axis=1)
df_grid_feature = df_grid_feature.rename(columns = {'ADM_SECT_C_x': 'ADM_SECT_C'})

gdf_cbds = gpd.GeoDataFrame(df_grid_feature, geometry = 'geometry_grids')
gdf_cbds['ADM_SECT_C'] = gdf_cbds['ADM_SECT_C'].astype(float).astype(int).astype(str) # 11110.0 -> '11110'
ref_col = ['ADM_SECT_C', 'idx']
geo_col = ['geometry_grids']
var_col = ['platarea', 'archarea', 'bcrat',\
       'totarea', 'vlrat', 'heit', 'grndflrcnt', 'ugrndflrcnt',\
       'rideuseelvtcnt', 'emgenuseelvtcnt', 'useaprday']
y_col = ['2019', '2020', '2021', '2022']

gdf_scbd = gdf_cbds[gdf_cbds['ADM_SECT_C'].isin(c.SCBD_CDS)]
gdf_gbd = gdf_cbds[gdf_cbds['ADM_SECT_C'].isin(c.GBD_CDS)]
gdf_ybd = gdf_cbds[gdf_cbds['ADM_SECT_C'] == c.YDPG_CD]


### 2. Get Seoul Boundary

In [3]:
seoul_shapefile = os.path.join(c.SSD_PATH, 'nsdi/LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11.shp')
seoul = gpd.read_file(seoul_shapefile)
seoul = seoul[['ADM_SECT_C', 'geometry']]
scbd_boundary = seoul[seoul['ADM_SECT_C'].isin(c.SCBD_CDS)]
gbd_boundary = seoul[seoul['ADM_SECT_C'].isin(c.GBD_CDS)]
ybd_boundary = seoul[seoul['ADM_SECT_C'] == c.YDPG_CD]

### 3. Save dfs

In [4]:
gdf_cbds.to_csv('../asset/preprocess/grid/gdf_cbds.csv')
gdf_cbds.head(20).to_csv('../asset/preprocess/grid/gdf_cbds_head.csv')
scbd_boundary.to_csv('../asset/preprocess/grid/scbd_boundary.csv')
gbd_boundary.to_csv('../asset/preprocess/grid/gbd_boundary.csv')
ybd_boundary.to_csv('../asset/preprocess/grid/ybd_boundary.csv')
gdf_cbds.head(10)

,pnu,rlgnm,rlgcd,sigungunm,lawdongnm,linm,bun,ji,naroadnm,naroadcd,...,2019,2020,2021,2022,x_5179,y_5179,geometry,grid_idx,ADM_SECT_C,geometry_grids
0,1168011800109590021,서울특별시,11,강남구,도곡동,NaN,959,21,남부순환로361길,116804166092,...,7996000,8398000,8965000,10060000,959090.778009,1.943028e+06,POINT (959090.778 1943027.542),36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
1,1168011800109590020,서울특별시,11,강남구,도곡동,NaN,959,20,남부순환로361길,116804166092,...,8885000,9331000,9845000,11050000,959074.037593,1.943033e+06,POINT (959074.038 1943032.991),36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
2,1168011800109200002,서울특별시,11,강남구,도곡동,NaN,920,2,남부순환로361길,116804166092,...,10320000,11140000,11810000,13280000,959117.186190,1.943028e+06,POINT (959117.186 1943027.622),36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
3,1168011800109190011,서울특별시,11,강남구,도곡동,NaN,919,11,남부순환로363길,116804166093,...,9006000,9717000,10190000,11310000,959128.096726,1.943046e+06,POINT (959128.097 1943046.156),36381,11680,"POLYGON ((959035.230 1942965.548, 959035.230 1..."
4,1168011800109590016,서울특별시,11,강남구,도곡동,NaN,959,16,남부순환로359길,116804166091,...,9526000,10000000,10450000,11740000,959036.318488,1.943066e+06,POINT (959036.318 1943065.615),36382,11680,"POLYGON ((959035.230 1943065.548, 959035.230 1..."
5,1168011800109240010,서울특별시,11,강남구,도곡동,NaN,924,10,남부순환로363길,116804166093,...,11140000,12020000,12610000,14180000,959134.541806,1.943103e+06,POINT (959134.542 1943103.095),36382,11680,"POLYGON ((959035.230 1943065.548, 959035.230 1..."
6,1168011800109240002,서울특별시,11,강남구,도곡동,NaN,924,2,남부순환로363길,116804166093,...,11140000,12020000,12610000,14180000,959133.787873,1.943115e+06,POINT (959133.788 1943114.803),36382,11680,"POLYGON ((959035.230 1943065.548, 959035.230 1..."
7,1168011800109590014,서울특별시,11,강남구,도곡동,NaN,959,14,남부순환로359길,116804166091,...,9160000,9620000,10150000,11400000,959026.325572,1.943083e+06,POINT (959026.326 1943083.362),36326,11680,"POLYGON ((958935.230 1943065.548, 958935.230 1..."
8,1168011800109590013,서울특별시,11,강남구,도곡동,NaN,959,13,남부순환로359길,116804166091,...,9160000,9620000,10150000,11400000,959021.808364,1.943092e+06,POINT (959021.808 1943091.582),36326,11680,"POLYGON ((958935.230 1943065.548, 958935.230 1..."
9,1168011800109590011,서울특별시,11,강남구,도곡동,NaN,959,11,남부순환로359길,116804166091,...,9343000,9620000,10150000,11400000,959008.378890,1.943115e+06,POINT (959008.379 1943115.006),36326,11680,"POLYGON ((958935.230 1943065.548, 958935.230 1..."


In [5]:
ybd_boundary.head(10)

,ADM_SECT_C,geometry
6,11560,"POLYGON ((945280.001 1950949.319, 945361.246 1..."


In [6]:
# Read gdf_cbds
gdf_cbds = pd.read_csv('../asset/preprocess/grid/gdf_cbds.csv', index_col = 0)
gdf_cbds.head(10)

,pnu,rlgnm,rlgcd,sigungunm,lawdongnm,linm,bun,ji,naroadnm,naroadcd,...,2019,2020,2021,2022,x_5179,y_5179,geometry,grid_idx,ADM_SECT_C,geometry_grids
0,1168011800109590021,서울특별시,11,강남구,도곡동,NaN,959,21,남부순환로361길,116804166092,...,7996000,8398000,8965000,10060000,959090.778009,1.943028e+06,POINT (959090.7780094582 1943027.54165114),36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
1,1168011800109590020,서울특별시,11,강남구,도곡동,NaN,959,20,남부순환로361길,116804166092,...,8885000,9331000,9845000,11050000,959074.037593,1.943033e+06,POINT (959074.0375926002 1943032.991403951),36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
2,1168011800109200002,서울특별시,11,강남구,도곡동,NaN,920,2,남부순환로361길,116804166092,...,10320000,11140000,11810000,13280000,959117.186190,1.943028e+06,POINT (959117.1861895586 1943027.621584438),36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
3,1168011800109190011,서울특별시,11,강남구,도곡동,NaN,919,11,남부순환로363길,116804166093,...,9006000,9717000,10190000,11310000,959128.096726,1.943046e+06,POINT (959128.096725813 1943046.15622537),36381,11680,"POLYGON ((959035.230213883 1942965.54807064, 9..."
4,1168011800109590016,서울특별시,11,강남구,도곡동,NaN,959,16,남부순환로359길,116804166091,...,9526000,10000000,10450000,11740000,959036.318488,1.943066e+06,POINT (959036.3184876485 1943065.615017564),36382,11680,"POLYGON ((959035.230213883 1943065.54807064, 9..."
5,1168011800109240010,서울특별시,11,강남구,도곡동,NaN,924,10,남부순환로363길,116804166093,...,11140000,12020000,12610000,14180000,959134.541806,1.943103e+06,POINT (959134.5418061555 1943103.095127398),36382,11680,"POLYGON ((959035.230213883 1943065.54807064, 9..."
6,1168011800109240002,서울특별시,11,강남구,도곡동,NaN,924,2,남부순환로363길,116804166093,...,11140000,12020000,12610000,14180000,959133.787873,1.943115e+06,POINT (959133.7878733623 1943114.803188868),36382,11680,"POLYGON ((959035.230213883 1943065.54807064, 9..."
7,1168011800109590014,서울특별시,11,강남구,도곡동,NaN,959,14,남부순환로359길,116804166091,...,9160000,9620000,10150000,11400000,959026.325572,1.943083e+06,POINT (959026.3255721567 1943083.362091733),36326,11680,"POLYGON ((958935.230213883 1943065.54807064, 9..."
8,1168011800109590013,서울특별시,11,강남구,도곡동,NaN,959,13,남부순환로359길,116804166091,...,9160000,9620000,10150000,11400000,959021.808364,1.943092e+06,POINT (959021.8083642247 1943091.582464794),36326,11680,"POLYGON ((958935.230213883 1943065.54807064, 9..."
9,1168011800109590011,서울특별시,11,강남구,도곡동,NaN,959,11,남부순환로359길,116804166091,...,9343000,9620000,10150000,11400000,959008.378890,1.943115e+06,POINT (959008.3788900722 1943115.005506424),36326,11680,"POLYGON ((958935.230213883 1943065.54807064, 9..."


In [7]:
# Read {}_boundary
ybd_boundary = pd.read_csv('../asset/preprocess/grid/ybd_boundary.csv', index_col = 0)
ybd_boundary.head(10)

,ADM_SECT_C,geometry
6,11560,"POLYGON ((945280.001025055 1950949.31881145, 9..."
